<a href="https://colab.research.google.com/github/yezzzzin/object-detectinon/blob/main/AiProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 라이브러리 import

In [9]:
!pip install torch torchvision

In [10]:
import pandas as pd
import torch
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2. 데이터 로드 & 전처리

In [12]:
# 데이터 로드 및 분할
data = pd.read_csv('/content/drive/MyDrive/dataset.csv')

train_data = pd.DataFrame()
test_data = pd.DataFrame()

for label in data['라벨'].unique():
    label_data = data[data['라벨'] == label]
    train_label_data, test_label_data = train_test_split(label_data, test_size=200, random_state=42)  # Splitting
    train_data = train_data.append(train_label_data)
    test_data = test_data.append(test_label_data)

train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

print(train_data[:10])

                                               데이터위치        라벨
0  /content/drive/MyDrive/img/squirrel/squirrel.6...  squirrel
1      /content/drive/MyDrive/img/lamb/lamb.955.jpeg      lamb
2  /content/drive/MyDrive/img/elephant/elephant.6...  elephant
3      /content/drive/MyDrive/img/lamb/lamb.630.jpeg      lamb
4        /content/drive/MyDrive/img/lamb/lamb.22.jpg      lamb
5  /content/drive/MyDrive/img/elephant/elephant.3...  elephant
6    /content/drive/MyDrive/img/horse/horse.563.jpeg     horse
7    /content/drive/MyDrive/img/horse/horse.344.jpeg     horse
8  /content/drive/MyDrive/img/elephant/elephant.3...  elephant
9  /content/drive/MyDrive/img/squirrel/squirrel.1...  squirrel


<ipython-input-12-408b67e455ef>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train_label_data)
<ipython-input-12-408b67e455ef>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test_label_data)
<ipython-input-12-408b67e455ef>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train_label_data)
<ipython-input-12-408b67e455ef>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test_label_data)
<ipython-input-12-408b67e455ef>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

In [13]:
# 사용자 정의 데이터셋 클래스
class AnimalDataset(Dataset):
    def __init__(self, dataframe, label_map, transform=None):
        self.dataframe = dataframe
        self.label_map = label_map
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx, 0]
        image = Image.open(image_path).convert("RGB")  # tensor 단위 오류 해결 위해 명시
        label_name = self.dataframe.iloc[idx, 1]
        label = self.label_map[label_name]

        if self.transform:
            image = self.transform(image)

        return image, label

# 라벨을 정수로 매핑 - 추후 label을 숫자가 아닌 글자로 반환하도록 해주는 딕셔너리
label_map = {'cat': 0, 'cow': 1, 'dog': 2, 'elephant': 3, 'horse': 4, 'lamb': 5, 'squirrel': 6}

# 이미지 전처리를 위한 transform 함수 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 및 데이터 로더 생성
train_dataset = AnimalDataset(train_data, label_map, transform=transform)  # 인스턴스는 init 파라미터 따름
test_dataset = AnimalDataset(test_data, label_map, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

3. 훈련

In [14]:
# 모델 정의
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)  # imageNet 데이터로 훈련된 모델 불러옴

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)  # 7종의 동물 클래스
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 학습 함수
def train_model(model, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        # 진행률 표시를 위해 tqdm을 사용
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 GPU로 이동
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader.dataset)
        tqdm.write(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
    return model

In [15]:
# 모델 학습
trained_model = train_model(model, criterion, optimizer, num_epochs=20)

Epoch 1/20: 100%|██████████| 700/700 [00:58<00:00, 11.88it/s]


Epoch 1/20, Loss: 0.0511


Epoch 2/20: 100%|██████████| 700/700 [00:58<00:00, 11.93it/s]


Epoch 2/20, Loss: 0.0203


Epoch 3/20: 100%|██████████| 700/700 [00:58<00:00, 12.00it/s]


...

Epoch 18/20, Loss: 0.0049


Epoch 19/20: 100%|██████████| 700/700 [00:59<00:00, 11.78it/s]


Epoch 19/20, Loss: 0.0033


Epoch 20/20: 100%|██████████| 700/700 [01:01<00:00, 11.47it/s]

Epoch 20/20, Loss: 0.0026


In [16]:
# 테스트 함수
def test_model_accuracy(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # 모델을 적절한 디바이스로 이동

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # 데이터를 GPU로 이동
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'테스트 데이터에 대한 모델 정확도: {accuracy:.2f}%')

def test_model_f1_confusion_matrix(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():  # 그래디언트 계산을 비활성화
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

    # 혼동 행렬 계산
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)

    # F1 Score 계산
    f1 = f1_score(all_labels, all_preds, average='weighted')
    print(f"F1 Score: {f1:.4f}")

In [17]:
test_model_accuracy(trained_model, test_loader)

테스트 데이터에 대한 모델 정확도: 94.21%


In [18]:
test_model_f1_confusion_matrix(trained_model, test_loader, device)

Confusion Matrix:
[[193   0   5   0   0   1   1]
 [  0 186   2   2   3   5   2]
 [  6   1 190   0   2   0   1]
 [  0   5   0 192   0   2   1]
 [  0  15   1   3 176   3   2]
 [  1   3   3   1   2 189   1]
 [  4   0   1   1   0   1 193]]
F1 Score: 0.9421
